In [1]:
# Import packages

import sqlite3
import requests
import pandas as pd

In [2]:
# connect to database 
conn = sqlite3.connect('./mock_resq.db')
cursor = conn.cursor()

In [3]:
# Execute SQL query to fetch unique years from the createdAt column
cursor.execute("SELECT DISTINCT strftime('%Y', CreatedAt) AS Year FROM orders")

# Fetch the results
unique_years = cursor.fetchall()

In [5]:
"""
This function retun holidays of the year by getting them from date.nager.at api
input:
    year as string like: '2022'
return:
    list of holiday days:
    ['2023-01-01', '2023-01-06', '2023-04-07']
"""

def get_holidays(year):
    response = requests.get(f'https://date.nager.at/api/v3/publicholidays/{year}/fi')
    if response.status_code == 200:
        holidays_data = response.json()
        return [holiday['date'] for holiday in holidays_data]
    else:
        raise 'site not response'

In [6]:
"""
This function creates a DataFrame that contains holiday dates for each year in the input list.

Args:
    unique_years (list): A list of unique years for which to get holiday dates.

Returns:
    all_holiday_dates (DataFrame): A DataFrame containing all holiday dates for the given years.
"""

def create_holiday_date(unique_years):
    # Get the number of unique years
    n = len(unique_years)
    
    # Initialize an empty DataFrame to store all holiday dates
    all_holiday_dates = pd.DataFrame()
    
    # Loop over each unique year
    for i in range(n):
        # Get the holiday dates for the current year
        holiday_dates = get_holidays(unique_years[i][0])
        
        # Create a DataFrame for the current year's holiday dates
        year_holiday_dates = pd.DataFrame({'holyDate': holiday_dates})
        
        # Concatenate the current year's holiday dates to the DataFrame of all holiday dates
        all_holiday_dates = pd.concat([all_holiday_dates, year_holiday_dates])
        
    # Return the DataFrame of all holiday dates
    all_holiday_dates.to_csv("all_holiday_dates.csv")
    return all_holiday_dates
